In [57]:
import pandas as pd
import numpy as np

In [58]:
#Import datafiles
links = pd.read_csv(r'C:\Users\shubham\Downloads\ml-latest-small\ml-latest-small\links.csv')
movies = pd.read_csv(r'C:\Users\shubham\Downloads\ml-latest-small\ml-latest-small\movies.csv')
ratings = pd.read_csv(r'C:\Users\shubham\Downloads\ml-latest-small\ml-latest-small\ratings.csv')
tags = pd.read_csv(r'C:\Users\shubham\Downloads\ml-latest-small\ml-latest-small\tags.csv')

In [59]:
Combine_rating = pd.merge(ratings, movies, on='movieId')


In [60]:
Combine_rating = Combine_rating.drop(columns = ['timestamp', 'genres'], axis=1)
Combine_rating.head(10)

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)
5,18,1,3.5,Toy Story (1995)
6,19,1,4.0,Toy Story (1995)
7,21,1,3.5,Toy Story (1995)
8,27,1,3.0,Toy Story (1995)
9,31,1,5.0,Toy Story (1995)


In [61]:
Combine_rating = Combine_rating.dropna(subset = ['title'])
movie_Rating_count = (Combine_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'RatingCount_total'})
     [['title', 'RatingCount_total']]
    )
movie_Rating_count.head(5)

,title,RatingCount_total
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [62]:
totalRatingCount_rating = Combine_rating.merge(movie_Rating_count, left_on = 'title', right_on = 'title', how = 'right')
totalRatingCount_rating.head(5)

,userId,movieId,rating,title,RatingCount_total
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [63]:
UserRating = totalRatingCount_rating.drop_duplicates(['userId','title'])
UserRating.head(5)

,userId,movieId,rating,title,RatingCount_total
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [64]:
movie_pivot = UserRating.pivot(index = 'userId', columns = 'title', values = 'rating').fillna(0)
movie_pivot.head(10)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [65]:
X = movie_pivot.values.T
X.shape

(9719, 610)

In [66]:
import sklearn
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=12, random_state=17)
matrix = SVD.fit_transform(X)
matrix.shape

(9719, 12)

In [67]:
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)
corr = np.corrcoef(matrix)
corr.shape

(9719, 9719)

In [68]:
movie_title = movie_pivot.columns
movie_title_list = list(movie_title)
coffey_hands = movie_title_list.index("Guardians of the Galaxy (2014)")

In [69]:
corr_coffey_hands  = corr[coffey_hands]
list(movie_title[(corr_coffey_hands >= 0.97)])

['Avengers, The (2012)',
 'Big Hero 6 (2014)',
 'Captain America: The First Avenger (2011)',
 'Deadpool (2016)',
 'Despicable Me (2010)',
 'Edge of Tomorrow (2014)',
 'Guardians of the Galaxy (2014)',
 'Inside Out (2015)',
 'Sherlock Holmes: A Game of Shadows (2011)',
 'Skyfall (2012)',
 'Star Wars: Episode VII - The Force Awakens (2015)',
 'The Lego Movie (2014)',
 'X-Men: First Class (2011)']